# Setup

In [5]:
import pymupdf4llm
import pathlib

# Exploration using General PyMuPDF4LLM Methods

In [6]:
vivado_documentation_md = pymupdf4llm.to_markdown("vivado_documentation.pdf")

In [9]:
pathlib.Path("markdown_files/vivado_documentation.md").write_bytes(vivado_documentation_md.encode())

869140

# Exploration using PyMuPDF4LLM + LlamaIndex

In [10]:
md_read = pymupdf4llm.LlamaMarkdownReader()
vivado_documentation_llama = md_read.load_data("vivado_documentation.pdf")

Successfully imported LlamaIndex


In [12]:
# page 9 (index 8) has text + images so is challenging
print(vivado_documentation_llama[8].to_dict()['text'])

*Chapter 1:* Introduction

     - Appendix D: JTAG Cables and Devices Supported by hw_server

    Appendix F: Configuration Memory Support
### **Getting Started**

After successfully implementing your design, the next step is to run it in hardware by
programming the FPGA or ACAP and debugging the design in-system. All of the necessary
commands to perform programming of FPGAs and in-system debugging of the design are in the
**Program and Debug** section of the **Flow Navigator** in the Vivado [®] Integrated Design
Environment (IDE) (see the following figure).

*Figure 1:* **Program and Debug Section of the Flow Navigator Panel**

UG908 (v2022.1) April 26, 2022 [www.xilinx.com](https://www.xilinx.com)
[Send Feedback](https://www.xilinx.com/about/feedback/document-feedback.html?docType=User_Guides&docId=UG908&Title=%20Vivado%20Design%20Suite%20User%20Guide&releaseVersion=2022.1&docPage=9)
Vivado Design Suite User Guide: Programming and Debugging 9


-----
